In [1]:
import time
start=time.time()
import numpy as np
import os
import math
import scipy
import sys
import csv
import itertools
import librosa as lb
import librosa.display
from scipy.fftpack import dct
from scipy.signal import argrelextrema
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import tensorflow as tf
from IPython.display import Audio
from glob import glob

In [2]:
# Initializing the parameters

fs=12800 # 12800 Sampling Fequency
fmin=50 # Minimum frequency choosen 
B=36     # Number of bins per octave
k1=0     # Smallest bin corresponding to 50Hz(fmin)
k2=B*np.log2(1500/fmin)# Largest bin corresponding to 1500Hz(fmax)
k2=int(np.ceil(k2))
Nf=int(B*np.log2(fs/(2*fmin))) # Total number of bins in CQT
Top=30

In [7]:
# Raw Pitch Accuracy
def RPA(v,pred_Melody,pitch_labels,Nt):
    N=0
    D=0
    for i in range(Nt):
        if v[i]==1:
            if pred_Melody[i]!=0:
                N+=v[i]*Thres(M(pred_Melody[i])-M(pitch_labels[i]))
            D+=v[i]
    Raw_pitch_acc=(N/D)*100
    #print("RPA is {} %".format(Raw_pitch_acc))
    return Raw_pitch_acc

# Raw Chroma Accuracy
def RCA(v,pred_Melody,pitch_labels,Nt):
    N=0
    D=0
    for i in range(Nt):
        if v[i]==1:
            if pred_Melody[i]!=0:
                N+=v[i]*Thres(Angular(M(pred_Melody[i])-M(pitch_labels[i])))
            D+=v [i]
    Raw_chroma_acc=(N/D)*100
    #print("RCA is {} %".format(Raw_chroma_acc))
    return Raw_chroma_acc


def Peak(magCQT,Nt):  # Nt is time frames
    Peaks=np.zeros((k2+1,Nt))
    Top_10Fo=np.zeros((Top,Nt))
    for frame in range(Nt):
        Row=scipy.signal.find_peaks(magCQT[k1:k2+1,frame])[0]
        for i in range(len(Row)):
            Peaks[Row[i]][frame]=magCQT[Row[i]][frame]
    Sort=np.argsort(Peaks,axis=0)
    Peaks1=np.zeros((k2+1,Nt))
    for i in range(Nt):
        arr=Sort[k2+1-int(Top/2):,i]
        ind=arr[Peaks[arr,i]!=0]

        for j in ind:
            Peaks1[j][i]=magCQT[j][i]
            Peaks1[j+1][i]=magCQT[j+1][i]
            Peaks1[j-1][i]=magCQT[j-1][i]
    Sort=np.argsort(Peaks1,axis=0)
    Top_10Fo=Sort[k2+1-Top:,:]
    Top_10Fo=Top_10Fo.astype(int)
    return Top_10Fo


def M(f):
    f_ref=100
    return 12*np.log2(f/f_ref)

def Thres(a):
    if -0.5<a and a<0.5:
        t=1
    else:
        t=0
    return t

def Angular(a):
    x=a-12*int(a/12+0.5)
    return x

def RawPitch(F_predicted,F_ground):
    return Thres(M(F_predicted)-M(F_ground))

def ChromaPitch(F_predicted,F_ground):
    return Thres(Angular(M(F_predicted)-M(F_ground)))

def match(ind,gt,s,frame):
    c=0
    for p in ind:
        fo=fmin*2**(s[p]/B)
        if RPA([1],[fo],[gt],1)==100:
            c+=1
    if c>1:
        c=1
    return c

In [17]:
##TESTING ON VARIOUS DATASETS

from keras import backend as K
from keras.models import model_from_json 
import mir_eval as mr

def custom_activation(x):
    return K.l2_normalize(x,axis=1)



## Testing on adc2004 dataset

print('Testing on the dataset....')
# load json and create model
json_file = open('./Mir1k-train-adc-test/model_800_eqweight.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json,custom_objects={'custom_activation':custom_activation})
# load weights into new model
loaded_model.load_weights("./Mir1k-train-adc-test/model.h5")
print("Loaded model from disk")

data_dirs="../../../../../../../Datasets/adc2004/Wavfile"
audio_file = glob(data_dirs + '/*.wav')

rpaList=[]
rcaList=[]
rec_b=[] # Recall before
prec_b=[] # Precicion before
prec_a=[] # Precision after
precis=[] # Precision same as prec_a but estimated from other approach
rpamir = []
rcamir = []
oamir = []
for song in range(len(audio_file)):
    pathname = os.path.splitext(audio_file[song])[0]
    filename = os.path.basename(pathname)
    print('Filename: ', filename)
    query = audio_file[song]
    data_dirp = '../../../../../../../Datasets/adc2004/PitchLabel'
    file = data_dirp + '/' + filename + '.txt'
    gtv,pitch_labels = np.loadtxt(file)[:,0],np.loadtxt(file)[:,1]
   
    # path_song=direc_songs+'/'+list_songs[song]
    x,fs=lb.load(query,sr=fs,duration=5.0) # Audio
    dur = len(x)/fs
    gtv = gtv[gtv<dur]
    pitch_labels = pitch_labels[:len(gtv)]
#     print(len(gtt),len(pl))

    CQT = lb.cqt(y=x, sr=fs, fmin=fmin, n_bins=Nf,bins_per_octave=B, hop_length=128)
    CQT_mag=np.abs(CQT)
    CQT_mag =CQT_mag / np.linalg.norm(CQT_mag)
    Nt=CQT_mag.shape[1]

    if Nt>len(pitch_labels):
        mm = Nt-len(pitch_labels)
        Nt = Nt-mm
    
    v=np.where(pitch_labels>0,1,pitch_labels)
    Top_10Peaks=Peak(CQT_mag,Nt)
    ww=[]
    tot=0
    pred_Melody=np.zeros(Nt) 
    for frame in range(Nt):
        feature=[]
        s=[]
        coun=0
        for k in range(Top):
                shift=Top_10Peaks[k,frame]
                if shift!=0:
                    feature.append(np.roll(CQT_mag[:,frame],-shift))
                    s.append(shift)
        feature=np.array(feature)
        coun+=feature.shape[0]
        if coun!=0:
            s=np.array(s)
            y_pred = loaded_model.predict(feature)
            pred = list()
            for i in range(len(y_pred)):
                pred.append(np.argmax(y_pred[i]))
           
            pred=np.array(pred)
            ind=np.where(pred==0)
            if len(ind[0])!=0:
                if pitch_labels[frame]!=0:
                    ww.append(match(ind[0],pitch_labels[frame],s,frame))
                ii = np.argmax(y_pred[ind[0],0])
                score_ind= ind[0][ii]
                pred_Melody[frame]=fmin*2**(s[score_ind]/B)
            tot+=coun

    rf, ref_voicing = mr.melody.freq_to_voicing(pitch_labels,voicing=None)
    ef, est_voicing = mr.melody.freq_to_voicing(pred_Melody,voicing=None)

    ref_cent = mr.melody.hz2cents(pitch_labels)
    est_cent = mr.melody.hz2cents(pred_Melody)

    oa = mr.melody.overall_accuracy(ref_voicing,ref_cent,est_voicing,est_cent,cent_tolerance=50)
    rpavv = mr.melody.raw_pitch_accuracy(ref_voicing,ref_cent,est_voicing,est_cent,cent_tolerance=50)
    rcavv = mr.melody.raw_chroma_accuracy(ref_voicing,ref_cent,est_voicing,est_cent,cent_tolerance=50)
    print(rpavv,rcavv)

    rpamir.append(rpavv)
    rcamir.append(rcavv)
    oamir.append(oa)
rpamir = np.array(rpamir)
rcamir = np.array(rcamir)
oamir = np.array(oamir)
print(np.mean(rpamir),np.mean(rcamir),np.mean(oamir))

Testing on the dataset....
Loaded model from disk
0
Filename:  daisy1
500 500 500
0.9781931464174455 0.9813084112149533
1
Filename:  daisy2
500 500 500
0.96 0.96
2
Filename:  pop1
500 500 500
0.6031390134529148 0.6076233183856502
3
Filename:  pop2
500 500 500
0.6313131313131313 0.6515151515151515
4
Filename:  pop4
500 500 500
0.7960893854748603 0.7960893854748603
5
Filename:  pop3
500 500 500
0.6590389016018307 0.6681922196796338
6
Filename:  daisy4
500 500 500
0.9517102615694165 0.9537223340040242
0.7970691199756569 0.8026358314677534 0.6625714285714287


In [20]:
##TESTING ON VARIOUS DATASETS

from keras import backend as K
from keras.models import model_from_json 
import mir_eval as mr

def custom_activation(x):
    return K.l2_normalize(x,axis=1)



## Testing on mirex05 dataset

print('Testing on the dataset....')
# load json and create model
json_file = open('./Mir1k-train-adc-test/model_800_eqweight.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json,custom_objects={'custom_activation':custom_activation})
# load weights into new model
loaded_model.load_weights("./Mir1k-train-adc-test/model.h5")
print("Loaded model from disk")

data_dirs="../../../../../../../Datasets/mirex05/Wavfile"
audio_file = glob(data_dirs + '/*.wav')

rpaList=[]
rcaList=[]
rec_b=[] # Recall before
prec_b=[] # Precicion before
prec_a=[] # Precision after
precis=[] # Precision same as prec_a but estimated from other approach
rpamir = []
rcamir = []
oamir = []
for song in range(len(audio_file)):
    pathname = os.path.splitext(audio_file[song])[0]
    filename = os.path.basename(pathname)
    print('Filename: ', filename)
    query = audio_file[song]
    data_dirp = '../../../../../../../Datasets/mirex05/PitchLabel'
    file = data_dirp + '/' + filename + '.txt'
    gtv,pitch_labels = np.loadtxt(file)[:,0],np.loadtxt(file)[:,1]
   
    x,fs=lb.load(query,sr=fs,duration=5.0) # Audio
    dur = len(x)/fs
    gtv = gtv[gtv<dur]
    pitch_labels = pitch_labels[:len(gtv)]

    CQT = lb.cqt(y=x, sr=fs, fmin=fmin, n_bins=Nf,bins_per_octave=B, hop_length=128)
    CQT_mag=np.abs(CQT)
    CQT_mag =CQT_mag / np.linalg.norm(CQT_mag)
    Nt=CQT_mag.shape[1]

    if Nt>len(pitch_labels):
        mm = Nt-len(pitch_labels)
        Nt = Nt-mm
    
    v=np.where(pitch_labels>0,1,pitch_labels)
    Top_10Peaks=Peak(CQT_mag,Nt)
    ww=[]
    tot=0
    pred_Melody=np.zeros(Nt) 
    for frame in range(Nt):
        feature=[]
        s=[]
        coun=0
        for k in range(Top):
                shift=Top_10Peaks[k,frame]
                if shift!=0:
                    feature.append(np.roll(CQT_mag[:,frame],-shift))
                    s.append(shift)
        feature=np.array(feature)
        coun+=feature.shape[0]
        if coun!=0:
            s=np.array(s)
            y_pred = loaded_model.predict(feature)
            pred = list()
            for i in range(len(y_pred)):
                pred.append(np.argmax(y_pred[i]))
           
            pred=np.array(pred)
            ind=np.where(pred==0)
            if len(ind[0])!=0:
                if pitch_labels[frame]!=0:
                    ww.append(match(ind[0],pitch_labels[frame],s,frame))
                ii = np.argmax(y_pred[ind[0],0])
                score_ind= ind[0][ii]
                pred_Melody[frame]=fmin*2**(s[score_ind]/B)
            tot+=coun

    rf, ref_voicing = mr.melody.freq_to_voicing(pitch_labels,voicing=None)
    ef, est_voicing = mr.melody.freq_to_voicing(pred_Melody,voicing=None)

    ref_cent = mr.melody.hz2cents(pitch_labels)
    est_cent = mr.melody.hz2cents(pred_Melody)

    oa = mr.melody.overall_accuracy(ref_voicing,ref_cent,est_voicing,est_cent,cent_tolerance=50)
    rpavv = mr.melody.raw_pitch_accuracy(ref_voicing,ref_cent,est_voicing,est_cent,cent_tolerance=50)
    rcavv = mr.melody.raw_chroma_accuracy(ref_voicing,ref_cent,est_voicing,est_cent,cent_tolerance=50)
    print(rpavv,rcavv)

    rpamir.append(rpavv)
    rcamir.append(rcavv)
    oamir.append(oa)
rpamir = np.array(rpamir)
rcamir = np.array(rcamir)
oamir = np.array(oamir)
print(np.mean(rpamir),np.mean(rcamir),np.mean(oamir))

Testing on the dataset....
Loaded model from disk
0
Filename:  train03
500 500 500
0.7780898876404494 0.7865168539325843
1
Filename:  train01
500 500 500
0.9324324324324325 0.9351351351351351
2
Filename:  train06
500 500 500
0.6829268292682927 0.7012195121951219
3
Filename:  train08
500 500 500
0.9240121580547113 0.9331306990881459
4
Filename:  train02
500 500 500
0.9228395061728395 0.9228395061728395
5
Filename:  train04
500 500 500
0.7643312101910829 0.821656050955414
6
Filename:  train09
500 500 500
0.8557919621749409 0.8581560283687943
7
Filename:  train07
500 500 500
0.7522388059701492 0.7582089552238805
8
Filename:  train10
500 500 500
1.0 1.0
9
Filename:  train05
500 500 500
0.9193083573487032 0.9193083573487032
0.8531971149253602 0.8636171098420619 0.5962000000000001


In [23]:
##TESTING ON VARIOUS DATASETS

from keras import backend as K
from keras.models import model_from_json 
import mir_eval as mr

def custom_activation(x):
    return K.l2_normalize(x,axis=1)



## Testing on mirex05 dataset

print('Testing on the dataset....')
# load json and create model
json_file = open('./Mir1k-train-adc-test/model_800_eqweight.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json,custom_objects={'custom_activation':custom_activation})
# load weights into new model
loaded_model.load_weights("./Mir1k-train-adc-test/model.h5")
print("Loaded model from disk")

data_dirs="../../../../../../../Datasets/Mir1K/Wavfile"
audio_file = glob(data_dirs + '/*.wav')

rpaList=[]
rcaList=[]
rec_b=[] # Recall before
prec_b=[] # Precicion before
prec_a=[] # Precision after
precis=[] # Precision same as prec_a but estimated from other approach
rpamir = []
rcamir = []
oamir = []

for song in range(800,1000):
    pathname = os.path.splitext(audio_file[song])[0]
    filename = os.path.basename(pathname)
    print('Filename: ', filename)
    query = audio_file[song]
    data_dirp = '../../../../../../../Datasets/Mir1K/PitchLabel'
    file = glob(data_dirp + '/' + filename + '.pv')
    rf = file[0]
    data = open(rf,mode='r')
    gtv = np.array([])
    pitch_labels = np.array([])
    for x in data:
        ymm = x.strip().split()
        ym = [float(i) for i in ymm]
        gtv = np.append(gtv,ym[0])
        pitch_labels = np.append(pitch_labels,ym[1])
     
    x,fs=lb.load(query,sr=fs,duration=5.0) # Audio
    dur = len(x)/fs
    gtv = gtv[gtv<dur]
    pitch_labels = pitch_labels[:len(gtv)]

    CQT = lb.cqt(y=x, sr=fs, fmin=fmin, n_bins=Nf,bins_per_octave=B, hop_length=128)
    CQT_mag=np.abs(CQT)
    CQT_mag =CQT_mag / np.linalg.norm(CQT_mag)
    Nt=CQT_mag.shape[1]

    if Nt>len(pitch_labels):
        mm = Nt-len(pitch_labels)
        Nt = Nt-mm
    
    v=np.where(pitch_labels>0,1,pitch_labels)
    Top_10Peaks=Peak(CQT_mag,Nt)
    ww=[]
    tot=0
    pred_Melody=np.zeros(Nt) 
    for frame in range(Nt):
        feature=[]
        s=[]
        coun=0
        for k in range(Top):
                shift=Top_10Peaks[k,frame]
                if shift!=0:
                    feature.append(np.roll(CQT_mag[:,frame],-shift))
                    s.append(shift)
        feature=np.array(feature)
        coun+=feature.shape[0]
        if coun!=0:
            s=np.array(s)
            y_pred = loaded_model.predict(feature)
            pred = list()
            for i in range(len(y_pred)):
                pred.append(np.argmax(y_pred[i]))
           
            pred=np.array(pred)
            ind=np.where(pred==0)
            if len(ind[0])!=0:
                if pitch_labels[frame]!=0:
                    ww.append(match(ind[0],pitch_labels[frame],s,frame))
                ii = np.argmax(y_pred[ind[0],0])
                score_ind= ind[0][ii]
                pred_Melody[frame]=fmin*2**(s[score_ind]/B)
            tot+=coun

    rf, ref_voicing = mr.melody.freq_to_voicing(pitch_labels,voicing=None)
    ef, est_voicing = mr.melody.freq_to_voicing(pred_Melody,voicing=None)

    ref_cent = mr.melody.hz2cents(pitch_labels)
    est_cent = mr.melody.hz2cents(pred_Melody)

    oa = mr.melody.overall_accuracy(ref_voicing,ref_cent,est_voicing,est_cent,cent_tolerance=50)
    rpavv = mr.melody.raw_pitch_accuracy(ref_voicing,ref_cent,est_voicing,est_cent,cent_tolerance=50)
    rcavv = mr.melody.raw_chroma_accuracy(ref_voicing,ref_cent,est_voicing,est_cent,cent_tolerance=50)
    print(rpavv,rcavv)

    rpamir.append(rpavv)
    rcamir.append(rcavv)
    oamir.append(oa)
rpamir = np.array(rpamir)
rcamir = np.array(rcamir)
oamir = np.array(oamir)
print(np.mean(rpamir),np.mean(rcamir),np.mean(oamir))

Testing on the dataset....
Loaded model from disk
Filename:  davidson_2_02
0.8693333333333333 0.8693333333333333
Filename:  ariel_4_07
0.8693586698337292 0.8693586698337292
Filename:  tammy_1_07
0.9433551198257081 0.9520697167755992
Filename:  amy_6_04
0.8760563380281691 0.8816901408450705
Filename:  jmzen_2_06
0.8414985590778098 0.8472622478386167
Filename:  Ani_1_04
0.711864406779661 0.711864406779661
Filename:  geniusturtle_4_02
0.856492027334852 0.856492027334852
Filename:  stool_3_04
0.768025078369906 0.768025078369906
Filename:  Ani_3_01
0.8287937743190662 0.8599221789883269
Filename:  bug_3_09
0.7984084880636605 0.830238726790451
Filename:  amy_16_03
0.9122257053291536 0.9122257053291536
Filename:  khair_2_01
0.653968253968254 0.6571428571428571
Filename:  amy_3_07
0.8363636363636363 0.8571428571428571
Filename:  jmzen_2_10
0.8825622775800712 0.896797153024911
Filename:  leon_1_09
0.7982708933717579 0.8040345821325648
Filename:  stool_2_07
0.7881619937694704 0.838006230529595
Fi

KeyboardInterrupt: 